In [1]:
import pandas  as pd
import numpy as np
%matplotlib inline
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [2]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
# import numpy as np
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.dummy import DummyClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# from sklearn import metrics
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

In [3]:
weather = pd.read_csv('.ipynb_checkpoints\weatherAUS.csv')
# now that we imported the data lets grab a peek and understand all are columns
weather.head()

FileNotFoundError: [Errno 2] No such file or directory: '.ipynb_checkpoints\\weatherAUS.csv'

In [ ]:
print(f'The number of rows are {weather.shape[0] } and the number of columns are {weather.shape[1]}')

In [ ]:
weather.info()

In [ ]:
weather['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
weather['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)
weather.head(10)

In [ ]:
weather.isnull().sum()

In [ ]:
(weather.isnull().sum()/len(weather))*100

In [ ]:
weather['RainToday']=weather['RainToday'].fillna(weather['RainToday'].mode()[0])
weather['RainTomorrow']=weather['RainTomorrow'].fillna(weather['RainTomorrow'].mode()[0])
#Filling the missing values for continuous variables with mode
weather['WindDir9am'] = weather['WindDir9am'].fillna(weather['WindDir9am'].mode()[0])
weather['WindGustDir'] = weather['WindGustDir'].fillna(weather['WindGustDir'].mode()[0])
weather['WindDir3pm'] = weather['WindDir3pm'].fillna(weather['WindDir3pm'].mode()[0])

In [ ]:
weather['MinTemp']=weather['MinTemp'].fillna(weather['MinTemp'].mean())
weather['MaxTemp']=weather['MaxTemp'].fillna(weather['MaxTemp'].mean())
weather['Rainfall']=weather['Rainfall'].fillna(weather['Rainfall'].mean())
weather['Evaporation']=weather['Evaporation'].fillna(weather['Evaporation'].mean())
weather['Sunshine']=weather['Sunshine'].fillna(weather['Sunshine'].mean())
weather['WindGustSpeed']=weather['WindGustSpeed'].fillna(weather['WindGustSpeed'].mean())
weather['WindSpeed9am']=weather['WindSpeed9am'].fillna(weather['WindSpeed9am'].mean())
weather['WindSpeed3pm']=weather['WindSpeed3pm'].fillna(weather['WindSpeed3pm'].mean())
weather['Humidity9am']=weather['Humidity9am'].fillna(weather['Humidity9am'].mean())
weather['Humidity3pm']=weather['Humidity3pm'].fillna(weather['Humidity3pm'].mean())
weather['Pressure9am']=weather['Pressure9am'].fillna(weather['Pressure9am'].mean())
weather['Pressure3pm']=weather['Pressure3pm'].fillna(weather['Pressure3pm'].mean())
weather['Cloud9am']=weather['Cloud9am'].fillna(weather['Cloud9am'].mean())
weather['Cloud3pm']=weather['Cloud3pm'].fillna(weather['Cloud3pm'].mean())
weather['Temp9am']=weather['Temp9am'].fillna(weather['Temp9am'].mean())
weather['Temp3pm']=weather['Temp3pm'].fillna(weather['Temp3pm'].mean())

In [ ]:
(weather.isnull().sum()/len(weather))*100

In [ ]:
fig,ax =plt.subplots(1,2)

plt.figure(figsize=(20,20))
sns.countplot(data=weather,x='RainToday',ax=ax[0])
sns.countplot(data=weather,x='RainTomorrow',ax=ax[1])

In [ ]:
plt.figure(figsize=(13,13))
sns.heatmap(weather.corr(), cmap = 'Greens', annot=True, fmt=".2f")

In [ ]:
fig = plt.figure(figsize = (10,7))
plt.scatter(weather['Temp3pm'], weather['MaxTemp'], alpha = 0.1)

In [ ]:
fig = plt.figure(figsize = (10,7))
plt.scatter(weather['Pressure3pm'], weather['Pressure9am'], alpha = 0.1)

In [ ]:
weather = weather.drop(['Temp3pm'], axis = 1)
weather = weather.drop(['Pressure9am'], axis = 1)
weather = weather.drop(['Date'], axis = 1) #maybe?

# maybe drop more colls

In [ ]:

le = preprocessing.LabelEncoder()
weather['Location'] = le.fit_transform(weather['Location'])
weather['WindDir9am'] = le.fit_transform(weather['WindDir9am'])
weather['WindDir3pm'] = le.fit_transform(weather['WindDir3pm'])
weather['WindGustDir'] = le.fit_transform(weather['WindGustDir'])
weather.head(10)

In [ ]:
x = weather.drop(['RainTomorrow'], axis = 1)
y = weather['RainTomorrow']



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)
# test size was 0.2

In [ ]:
z = X_train.values #returns a numpy array
min_max_scaler = MinMaxScaler()
z_scaled = min_max_scaler.fit_transform(z)
X_train = pd.DataFrame(z_scaled)

z = X_test.values #returns a numpy array
z_scaled = min_max_scaler.fit_transform(z)
X_test = pd.DataFrame(z_scaled)
X_train.head()
# X_test.head()

In [ ]:
dummyModel = DummyClassifier(strategy="stratified")
dummyModel.fit(X_train, y_train)
predictionsDummy = dummyModel.predict(X_test)

accuracyDummy = metrics.accuracy_score(y_test, predictionsDummy)
accuracyDummy

In [ ]:
modelLogReg = LogisticRegression(max_iter=500)
modelLogReg.fit(X_train, y_train)
predictionsLogReg = modelLogReg.predict(X_test)
accuracyLogReg = modelLogReg.score(X_test, y_test)

conf = confusion_matrix(y_test, predictionsLogReg)
print ("The accuracy of Logistic Regression is : ", accuracyLogReg*100, "%")
print(accuracyLogReg)
# print("F1 score for logistic regression is :",f1_score(y_test, predictionsLogReg,)*100, "%")

In [ ]:
plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, predictionsLogReg), cmap = "Blues", annot=True, fmt = '.0f')

In [ ]:
maxi = 1 #saving the index of the highest score
max = 0 #saving the value of the highest score
# for i in range(2,20):
#     accuracy = cross_val_score(modelLogReg, X_train, y_train, cv = i).mean()
#     if (accuracy > max):
#         maxi = i
#         max = accuracy
# print('Best index:', maxi, "\ncross_val_score of index", maxi, ':', max)

accuracy = cross_val_score(modelLogReg, X_train, y_train, cv = 11).mean()
maxi = 11
max = accuracy
print('Best index:', maxi, "\ncross_val_score of index", maxi, ':', max)


# print("Best index: 11 
# cross_val_score of index 11 : 0.8424691109678968")

In [ ]:
maxi = 1 #saving the index of the highest score
max = 0 #saving the value of the highest score
# for i in range(1,20):
#     modelKNN = KNeighborsClassifier(n_neighbors = i, weights='distance')
#     modelKNN.fit(X_train, y_train)
#     accuracy = modelKNN.score(X_test, y_test)
#     if (accuracy > max):
#         maxi = i
#         max = accuracy
# print(maxi, "  ", max)

modelKNN = KNeighborsClassifier(n_neighbors = 12, weights='distance')
modelKNN.fit(X_train, y_train)
accuracy = modelKNN.score(X_test, y_test)
print("12,",accuracy)

In [ ]:
modelKNN = KNeighborsClassifier(n_neighbors = 9, weights='distance')
modelKNN.fit(X_train, y_train)
predictionsKNN = modelKNN.predict(X_test)
accuracyKNN = metrics.accuracy_score(y_test, predictionsKNN) #Resturns the same value as the score() method in the
#previous cell
accuracyKNN

conf = confusion_matrix(y_test, predictionsKNN)
print ("The accuracy of Logistic Regression is : ", accuracyKNN*100, "%")
print(accuracyLogReg)
# print("F1 score for logistic regression is :",f1_score(y_test, predictionsKNN,)*100, "%")

In [ ]:
plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, predictionsKNN), cmap = "Blues", annot=True, fmt = '.0f')

In [ ]:
maxi = 1 #saving the index of the highest score
max = 0 #saving the value of the highest score
# for i in range(2,20):
#     accuracy = cross_val_score(modelKNN, X_train, y_train, cv = i).mean()
#     if (accuracy > max):
#         maxi = i
#         max = accuracy
# print('Best index:', maxi, "\ncross_val_score of index", maxi, ':', max)

accuracy = cross_val_score(modelKNN, X_train, y_train, cv = 14).mean()
print("14,",accuracy)